In [7]:
import pandas as pd
import json
from data import get_data as g_d
from models import arima_model as a_m, ets_model as ets_m, theta_model as th_m

def create_forecast_json(crypto_df, model_fit, steps=30, model_name=''):
    if isinstance(crypto_df, pd.DataFrame):
        if 'price' not in crypto_df.columns:
            raise ValueError("The DataFrame does not contain a 'price' column.")
        crypto_df = crypto_df['price']
    
    if isinstance(crypto_df.index, pd.PeriodIndex):
        crypto_df.index = crypto_df.index.to_timestamp()
    
    if hasattr(model_fit, 'get_forecast'):  # ARIMA
        forecast_result = model_fit.get_forecast(steps=steps)
        forecast = forecast_result.predicted_mean
    else:  # ETS and Theta
        forecast = model_fit.forecast(steps=steps)

    if isinstance(forecast, pd.Series):
        forecast = forecast.iloc[1:]

    forecast_dates = pd.date_range(start=crypto_df.index[-1] + pd.Timedelta(hours=1), periods=steps-1, freq='h')    
    forecast_series = pd.Series(forecast.values, index=forecast_dates)
    
    combined_series = pd.concat([crypto_df, forecast_series])

    combined_data_dict = {'data': combined_series.tolist(), 'index': combined_series.index.strftime('%Y-%m-%dT%H:%M:%S').tolist(), 'model_name': model_name}
    
    json_str = json.dumps(combined_data_dict)
    
    return json_str

api_key = '7db9598cf3ebf01148dda37d500a35844ef435670b3d962a53f6d0a397c98d43'
hours = 720
crypto_ids = ['BNB', 'SOL', 'XRP']

crypto_dfs = {}
for crypto_id in crypto_ids:
    df = g_d.fetch_historical_data(crypto_id, hours, api_key)
    if 'date' in df.columns and not df.index._is_all_dates:
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        df = df.asfreq('h')
    crypto_dfs[crypto_id] = df


In [8]:
model_funcs = {
    'arima': a_m.fit_arima_model,
    'ets': ets_m.fit_ets_model,
    'theta': th_m.fit_theta_model
}

forecast_jsons = {}
for crypto_id, crypto_df in crypto_dfs.items():
    for model_name, model_func in model_funcs.items():
        # Fit the model
        model_fit = model_func(crypto_df)
        # Create the forecast JSON, including the model name
        forecast_key = f'{crypto_id}_{model_name}'
        forecast_jsons[forecast_key] = create_forecast_json(crypto_df['price'], model_fit, 30, model_name=forecast_key)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as sta

In [9]:
for key in forecast_jsons.keys():
    print(forecast_jsons[key])

{"data": [376.52, 376.68, 373.56, 374.88, 377.56, 371.56, 373.47, 374.87, 375.08, 377.54, 375.95, 375.47, 374.19, 375.56, 377.03, 372.15, 375.33, 377.59, 379.1, 379.73, 379.07, 379.67, 380.21, 380.45, 381.38, 380.56, 379.61, 380.04, 380.43, 379.53, 380.48, 381.74, 382.06, 382.83, 382.64, 382.51, 382.08, 381.65, 381.39, 380.55, 380.32, 379.53, 378.71, 379.75, 380.78, 380.66, 379.77, 379.56, 379.74, 383.95, 384.65, 384.07, 384.7, 385.04, 384.36, 386.17, 386.3, 389.03, 388.68, 388.59, 390.23, 388.53, 389.04, 385.28, 383.94, 384.23, 386.09, 385.48, 387.79, 386.11, 389.24, 385.07, 388.15, 389.54, 390.19, 394.0, 397.37, 397.14, 398.4, 398.35, 396.45, 398.44, 403.01, 401.88, 401.76, 401.86, 402.37, 400.84, 400.84, 399.21, 398.13, 398.88, 398.65, 396.15, 397.82, 399.08, 399.37, 398.55, 399.27, 398.04, 394.77, 393.03, 394.09, 392.85, 395.22, 395.58, 396.68, 395.62, 396.17, 394.88, 395.62, 395.47, 397.5, 395.92, 397.91, 398.24, 398.1, 400.9, 405.44, 412.87, 409.77, 406.67, 408.41, 413.8, 413.95,